In [4]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse import identity
from scipy.sparse import kron

L=2

# Define some operators

In [9]:
a_b = diags(np.sqrt(np.linspace(1,L-1,L-1)),offsets=1)
i_b = identity(L)
a_f = diags(np.sqrt(np.linspace(1,1,1)),offsets=1)

sz = diags([1.0,-1.0])
i_f = identity(2)
a_b.todense(), i_b.todense(), a_f.todense(), sz.todense()

(matrix([[0., 1.],
         [0., 0.]]),
 matrix([[1., 0.],
         [0., 1.]]),
 matrix([[0., 1.],
         [0., 0.]]),
 matrix([[ 1.,  0.],
         [ 0., -1.]]))

# Bosonic Hilbert space

In [8]:
N_bos = 6
product_list = [i_b] * N_bos  # only the identity for bosons repeated N_bos times
a_b_list = []  # this will contain a1...a6
for i in np.arange(0,N_bos):  # loop over all bosonic operators
    operator_list = product_list.copy()  # all elements are the identity operator
    operator_list[i] = a_b  # the i^th element is now the annihilation operator for a single boson
    a_b_list.append(operator_list[0]) # start taking tensor products from first element
    for a in operator_list[1:]:
        a_b_list[i] = kron(a_b_list[i],a)  # do the outer product between each operator_list element

In [14]:
a_b_list

[<64x64 sparse matrix of type '<class 'numpy.float64'>'
 	with 1024 stored elements (blocksize = 2x2) in Block Sparse Row format>,
 <64x64 sparse matrix of type '<class 'numpy.float64'>'
 	with 512 stored elements (blocksize = 2x2) in Block Sparse Row format>,
 <64x64 sparse matrix of type '<class 'numpy.float64'>'
 	with 512 stored elements (blocksize = 2x2) in Block Sparse Row format>,
 <64x64 sparse matrix of type '<class 'numpy.float64'>'
 	with 512 stored elements (blocksize = 2x2) in Block Sparse Row format>,
 <64x64 sparse matrix of type '<class 'numpy.float64'>'
 	with 512 stored elements (blocksize = 2x2) in Block Sparse Row format>,
 <64x64 sparse matrix of type '<class 'numpy.float64'>'
 	with 512 stored elements in COOrdinate format>]

# Fermionic Hilbert space

In [15]:
N_f = 3  # number of fermionic sites -> fixed for the supersymmetric model mini-BMN with 2 matrices 

In [16]:
product_list = [i_f] * N_f  # only the identity for fermions repeated N_f times
a_f_list = []  # this will contain f1...f3
for i in np.arange(0,N_f):  # loop over all bosonic operators
    operator_list = product_list.copy()  # all elements are the identity operator
    operator_list[i] = a_f  # the i^th element is now the annihilation operator for a single fermion
    for j in np.arange(0,i):  # the 0:(i-1) elements are replaced by sigma_Z
        operator_list[j] = sz
    a_f_list.append(operator_list[0])  # start taking tensor products from the first operator
    for a in operator_list[1:]:
        a_f_list[i] = kron(a_f_list[i],a)  # do the outer product 

In [17]:
a_f_list

[<8x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 16 stored elements (blocksize = 2x2) in Block Sparse Row format>,
 <8x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 8 stored elements (blocksize = 2x2) in Block Sparse Row format>,
 <8x8 sparse matrix of type '<class 'numpy.float64'>'
 	with 8 stored elements in COOrdinate format>]

# Combine Bosonic and Fermionic spaces

In [20]:
i_b_tot = identity(L**N_bos) 
i_f_tot = identity(2**N_f)

In [21]:
op_list = []
for a in a_b_list:
    op_list.append(kron(a,i_f_tot))
for a in a_f_list:
    op_list.append(kron(i_b_tot,a))

In [22]:
len(op_list), op_list

(9,
 [<512x512 sparse matrix of type '<class 'numpy.float64'>'
  	with 8192 stored elements in COOrdinate format>,
  <512x512 sparse matrix of type '<class 'numpy.float64'>'
  	with 4096 stored elements in COOrdinate format>,
  <512x512 sparse matrix of type '<class 'numpy.float64'>'
  	with 4096 stored elements in COOrdinate format>,
  <512x512 sparse matrix of type '<class 'numpy.float64'>'
  	with 4096 stored elements in COOrdinate format>,
  <512x512 sparse matrix of type '<class 'numpy.float64'>'
  	with 4096 stored elements in COOrdinate format>,
  <512x512 sparse matrix of type '<class 'numpy.float64'>'
  	with 4096 stored elements in COOrdinate format>,
  <512x512 sparse matrix of type '<class 'numpy.float64'>'
  	with 1024 stored elements in COOrdinate format>,
  <512x512 sparse matrix of type '<class 'numpy.float64'>'
  	with 512 stored elements in COOrdinate format>,
  <512x512 sparse matrix of type '<class 'numpy.float64'>'
  	with 512 stored elements in COOrdinate format>]

# Build the Hamiltonian

In [24]:
x_list = []
# only use the bosonic operators
bosons = op_list[:N_bos]
for op in bosons:
    x_list.append(1/np.sqrt(2)*(op.conjugate().transpose() + op))
    
# save the fermions operators for convenience in a new list
fermions = op_list[-N_f:]

In [25]:
# Create the simple quartic Hamiltonian.
H_k = 0

for a in op_list[:N_bos]:
    H_k = H_k + a.conjugate().transpose() * a

for a in op_list[-N_f:]:
    H_k = H_k + (3.0 / 2) * a.conjugate().transpose() * a

# vacuum energy
H_k = H_k + 0.25 * (2 * N_bos - 3 * N_f - 3)

In [27]:
from scipy.sparse.linalg import eigsh
eigv = eigsh(H_k,k=10,which='SA',return_eigenvectors=False,tol=0)
print(f"Free Hamiltonian eigenvalues: {eigv[::-1]}")

Free Hamiltonian eigenvalues: [2.07724264e-16 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.50000000e+00
 1.50000000e+00 1.50000000e+00]


In [32]:
#BOSON INTERACTION
V_b = (
    x_list[2] * x_list[2] * x_list[3] * x_list[3]
    + x_list[2] * x_list[2] * x_list[4] * x_list[4]
    + x_list[1] * x_list[1] * x_list[3] * x_list[3]
    + x_list[1] * x_list[1] * x_list[5] * x_list[5]
    + x_list[0] * x_list[0] * x_list[4] * x_list[4]
    + x_list[0] * x_list[0] * x_list[5] * x_list[5]
    - 2 * x_list[0] * x_list[2] * x_list[3] * x_list[5]
    - 2 * x_list[0] * x_list[1] * x_list[3] * x_list[4]
    - 2 * x_list[1] * x_list[2] * x_list[4] * x_list[5]
)
#BOSONS-FERMIONS INTERACTION
V_bf = (2j / np.sqrt(2)) * (
    (x_list[0] - 1j * x_list[3])
    * fermions[1].conjugate().transpose()
    * fermions[2].conjugate().transpose()
    + (x_list[1] - 1j * x_list[4])
    * fermions[2].conjugate().transpose()
    * fermions[0].conjugate().transpose()
    + (x_list[2] - 1j * x_list[5])
    * fermions[0].conjugate().transpose()
    * fermions[1].conjugate().transpose()
    - (x_list[0] + 1j * x_list[3]) * fermions[2] * fermions[1]
    - (x_list[1] + 1j * x_list[4]) * fermions[0] * fermions[2]
    - (x_list[2] + 1j * x_list[5]) * fermions[1] * fermions[0]
)

# g2N= 0.2

In [29]:
# t'Hooft coupling
g2N = 0.2
# Number of colors
N = 2
# full hamiltonian
H = H_k + g2N/N*V_b + np.sqrt(g2N/N)*V_bf

In [31]:
eigv = eigsh(H,k=10,which='SM',return_eigenvectors=False,tol=0)
print(f"Full Hamiltonian with lambda={g2N} : {eigv[::-1]}")

Full Hamiltonian with lambda=0.2 : [0.00328726 0.95978401 0.95978401 0.95978401 1.00188691 1.00188691
 1.00188691 1.59620668 1.59620668 1.59620668]


In [38]:
Hm = H.todense()
np.save('L2_l02_BMN_H.npy', Hm)

# g2N = 0.5

In [39]:
# t'Hooft coupling
g2N = 0.5
# Number of colors
N = 2
# full hamiltonian
H = H_k + g2N/N*V_b + np.sqrt(g2N/N)*V_bf

In [40]:
eigv = eigsh(H,k=10,which='SM',return_eigenvectors=False,tol=0)
#0.01690
print(f"Full Hamiltonian with lambda={g2N} : {eigv[::-1]}")

Full Hamiltonian with lambda=0.5 : [0.01690123 0.92239557 0.92239557 0.92239557 1.00677384 1.00677384
 1.00677384 1.68578715 1.72582874 1.72582874]


In [41]:
Hm = H.todense()
np.save('L2_l05_BMN_H.npy', Hm)

# g2N = 1.0

In [42]:
# t'Hooft coupling
g2N = 1.0
# Number of colors
N = 2
# full hamiltonian
H = H_k + g2N/N*V_b + np.sqrt(g2N/N)*V_bf

In [43]:
eigv = eigsh(H,k=10,which='SM',return_eigenvectors=False,tol=0)
#0.04829
print(f"Full Hamiltonian with lambda={g2N} : {eigv[::-1]}")

Full Hamiltonian with lambda=1.0 : [0.04829007 0.87933179 0.87933179 0.87933179 0.99982335 0.99982335
 0.99982335 1.4362856  1.58661088 1.58661088]


In [44]:
Hm = H.todense()
np.save('L2_l10_BMN_H.npy', Hm)

# g2N = 2.0

In [45]:
# t'Hooft coupling
g2N = 2.0
# Number of colors
N = 2
# full hamiltonian
H = H_k + g2N/N*V_b + np.sqrt(g2N/N)*V_bf

In [46]:
#0.08385
eigv = eigsh(H,k=10,which='SM',return_eigenvectors=False,tol=0)
print(f"Full Hamiltonian with lambda={g2N} : {eigv[::-1]}")

Full Hamiltonian with lambda=2.0 : [0.08385032 0.75061257 0.75061257 0.75061257 0.86896928 0.86896928
 0.86896928 1.01454483 1.21948269 1.21948269]


In [47]:
Hm = H.todense()
np.save('L2_l20_BMN_H.npy', Hm)